### Vanilla CTR Zeroshot

In [1]:
import pickle
import pandas as pd
import numpy as np
import os
import time
from dotenv import load_dotenv

In [2]:
from openai import OpenAI
load_dotenv()
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [3]:
user_train_path = '../train_valid_test/user_train.pkl'
user_valid_path = '../train_valid_test/user_valid.pkl'
user_test_path = '../train_valid_test/user_test.pkl'

In [4]:
with open(user_train_path, 'rb') as f:
    user_train_dict = pickle.load(f)

In [5]:
with open(user_valid_path, 'rb') as f:
    user_valid_dict = pickle.load(f)

In [6]:
with open(user_test_path, 'rb') as f:
    user_test_dict = pickle.load(f)

In [7]:
user_id = 1
print(user_train_dict[user_id])
print(user_valid_dict[user_id])
print(user_test_dict[user_id])

[[1961, 2028, 3105, 938, 1962, 1035, 150, 2018, 1028, 1097, 914, 2797, 1287, 2762, 1246, 661, 2918, 531, 3114, 2791, 1029, 2321, 1197, 594, 2398, 1545, 527, 1, 588, 2687, 745, 595, 2294, 783, 2355, 1907]]
[1566]
[48]


In [8]:
len(user_train_dict[user_id][0])

36

In [9]:
user_profile_path = '../user_profile/cleaned_user_profile_mixtral.pkl'
with open(user_profile_path, 'rb') as f:
    user_profile_dict = pickle.load(f)
print(user_profile_dict[user_id])


            The user has a diverse taste in movies, with a preference for thought-provoking and emotionally engaging films. They enjoy dramas that explore complex themes such as mental illness, racial injustice, and the meaning of life. The user is drawn to movies that challenge their perspective and provoke deep thought, often favoring those that tackle serious and mature subjects.
            
            In addition to dramas, the user also has a soft spot for classic Disney films, as evidenced by their fondness for Cinderella. They appreciate the enchanting storytelling and timeless charm of these animated tales.
            
            The user's taste in movies also extends to comedies, as seen in their appreciation for the hilarious antics of Wallace & Gromit and the dark humor of Fargo. They are also drawn to adventure films, as demonstrated by their love for Star Wars: Episode IV - A New Hope and Back to the Future.
            
            However, the user is not a fan of 

In [10]:
movie_summary_df = pd.read_csv('../movie_summary_25_words.csv')
movie_summary_df.head()

,movie_id,name,cleaned_genre,year,summary
0,1,Toy Story,"Animation,Children's,Comedy",1995,Toy Story is a groundbreaking animated film th...
1,2,Jumanji,"Adventure,Children's,Fantasy",1995,"""Jumanji (1995) is a thrilling adventure film ..."
2,3,Grumpier Old Men,"Comedy,Romance",1995,"""Grumpier Old Men is a hilarious and heartwarm..."
3,4,Waiting to Exhale,"Comedy,Drama",1995,"""Waiting to Exhale"" (1995) is a heartfelt dram..."
4,5,Father of the Bride Part II,Comedy,1995,Father of the Bride Part II (1995) is a heartw...


In [11]:
user_df = pd.read_csv('../users.csv')
user_df.head()

,user_id,user_profile
0,1,female k-12 student under 18 years of age
1,2,male self-employed above 56 years of age
2,3,male scientist between 25 to 34 years of age
3,4,male executive/managerial between 45 to 49 yea...
4,5,male writer between 25 to 34 years of age


In [12]:
ratings_df = pd.read_csv('../ratings.csv')
ratings_df.head()

,user_id,movie_id,rating
0,1,3186,4
1,1,1721,4
2,1,1022,5
3,1,1270,5
4,1,2340,3


In [15]:
crt_vanilla_valid_zeroshot_dataset = dict()
cnt = 0
for user, user_profile in user_profile_dict.items():
    # if cnt < 4578:
    #     cnt += 1
    #     continue
    cnt += 1
    # print(f"User: {user}")
    prompt = """
    You are an expert movie critic. You are provided with the user profile and list of recent movies that the user has watched and whether the user likes it or not.
    User Profile -
    """
    ### Create the user profile
    final_user_profile = "The user is a "+user_df[user_df['user_id'] == user]['user_profile'].values[0] + "\n"
    # print(final_user_profile)
    prompt += final_user_profile
    
    prompt += """
   User watched the following movies recently in the given order. List of recent movies -
    """
    # print(prompt)
    ### Create the like/dislike list of last 20 movies
    # print(user_train_dict[user][0][-20:])
    user_rating_dict = ratings_df[ratings_df['user_id'] == user]
    # print(user_rating_dict)
    liked_movies = []
    disliked_movies = []
    for movie in user_train_dict[user][0][-20:]:
        user_movie_rating = user_rating_dict[user_rating_dict['movie_id'] == movie]['rating'].values[0]
        # print(movie, user_movie_rating)
        movie_details = movie_summary_df[movie_summary_df['movie_id'] == movie]
        if user_movie_rating >= 3:
            if movie != user_train_dict[user][0][-20:][-1]:
                prompt += "\nLiked " + movie_details['name'].values[0] + ","
            else:
                ### For the last movie - ending with a . and starting new line
                prompt += "\nLiked " + movie_details['name'].values[0] + "\n"
        else:
            if movie != user_train_dict[user][0][-20:][-1]:
                prompt += "\nDisliked " + movie_details['name'].values[0] + ","
            else:
                ### For the last movie - ending with a . and starting new line
                prompt += "\nDisliked " + movie_details['name'].values[0] + "\n"

    ### To predict
    # prompt += """
    # Predict whether the user will like or dislike the next movie -
    # """
    # print(user_valid_dict[user][0])
    movie_details = movie_summary_df[movie_summary_df['movie_id'] == user_valid_dict[user][0]]
    # print("Test Movie Rating:", user_rating_dict[user_rating_dict['movie_id'] == user_valid_dict[user][0]]['rating'].values[0])
    # prompt += movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " +  movie_details['summary'].values[0]
    prompt += f"""
    Predict whether the user will like or dislike the next movie - {movie_details['name'].values[0]}. Answer in the following format -
    Prediction: Yes or No
    """
    # print(f"Prompt length: {len(prompt.split())}", prompt)
    crt_vanilla_valid_zeroshot_dataset[user] = prompt
    # break

In [16]:
print(len(crt_vanilla_valid_zeroshot_dataset))

6040


In [17]:
crt_vanilla_valid_zeroshot_dataset[3246]

"\n    You are an expert movie critic. You are provided with the user profile and list of recent movies that the user has watched and whether the user likes it or not.\n    User Profile -\n    The user is a female academic/educator between 35 to 44 years of age\n\n   User watched the following movies recently in the given order. List of recent movies -\n    \nLiked Clear and Present Danger ,\nDisliked Twister ,\nDisliked Air Force One ,\nLiked Ghost and the Darkness, The ,\nDisliked Fifth Element, The ,\nLiked Maverick ,\nDisliked Single White Female ,\nDisliked World Is Not Enough, The ,\nDisliked Days of Thunder ,\nDisliked Beverly Hills Cop III ,\nLiked Replacement Killers, The ,\nLiked U.S. Marshalls ,\nDisliked Dante's Peak ,\nDisliked Hot Shots! Part Deux ,\nDisliked River Wild, The ,\nLiked Cliffhanger ,\nLiked Air America ,\nLiked Dick Tracy ,\nDisliked Inspector Gadget ,\nDisliked Last Action Hero \n\n    Predict whether the user will like or dislike the next movie - Blues Bro

In [18]:
crt_valid_zeroshot_dataset_path = './ctr_vanilla_zeroshot_dataset/ctr_vanilla_valid_zeroshot_dataset.pkl'
with open(crt_valid_zeroshot_dataset_path, 'wb') as f:
    pickle.dump(crt_vanilla_valid_zeroshot_dataset, f)
    f.close()

In [37]:
# with open(crt_valid_zeroshot_dataset_path, 'rb') as f:
#     crt_valid_zeroshot_dataset_temp = pickle.load(f)
# print(len(crt_valid_zeroshot_dataset_temp), crt_valid_zeroshot_dataset_temp[1])

In [19]:
content = prompt
print(content)
print('*'*100)
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "user",
      "content": content
    }
  ],
  temperature=0,
  max_tokens=1024,
  top_p=0.75
)


response = response.choices[0].message.content
print(response)
len(response.split(' '))


    You are an expert movie critic. You are provided with the user profile and list of recent movies that the user has watched and whether the user likes it or not.
    User Profile -
    The user is a male scientist between 25 to 34 years of age

   User watched the following movies recently in the given order. List of recent movies -
    
Disliked Last Man Standing ,
Liked Juror, The ,
Liked Walk in the Clouds, A ,
Liked Sommersby ,
Disliked White Man's Burden ,
Liked Practical Magic ,
Liked It Could Happen to You ,
Disliked Your Friends and Neighbors ,
Liked General's Daughter, The ,
Disliked We're No Angels ,
Disliked Out of Africa ,
Liked Powder ,
Liked Instinct ,
Disliked Corruptor, The ,
Liked Where the Heart Is ,
Liked Jack ,
Disliked Evening Star, The ,
Liked Preacher's Wife, The ,
Liked Indecent Proposal ,
Liked 200 Cigarettes 

    Predict whether the user will like or dislike the next movie - Showgirls . Answer in the following format -
    Prediction: Yes or No
    
*****

2